In [1]:
%pip install -r requirements.txt
%pip install transformers
%pip install torch
%pip install tensorflow.keras

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
ERROR: Could not find a version that satisfies the requirement tensorflow.keras (from versions: none)
ERROR: No matching distribution found for tensorflow.keras
Note: you may need to restart the kernel to use updated packages.


*Data Loading*

In [2]:
import json

def load_metadata(metadata_path):
    with open(metadata_path, 'r') as f:
        metadata = json.load(f)
    return metadata


metadata_path = "/Users/lakshya/Desktop/Projects/VeriFace/Dataset/metadata.json"
metadata = load_metadata(metadata_path)
print(metadata)  

{'aagfhgtpmv.mp4': {'label': 'FAKE', 'split': 'train', 'original': 'vudstovrck.mp4'}, 'aapnvogymq.mp4': {'label': 'FAKE', 'split': 'train', 'original': 'jdubbvfswz.mp4'}, 'abarnvbtwb.mp4': {'label': 'REAL', 'split': 'train', 'original': None}, 'abofeumbvv.mp4': {'label': 'FAKE', 'split': 'train', 'original': 'atvmxvwyns.mp4'}, 'abqwwspghj.mp4': {'label': 'FAKE', 'split': 'train', 'original': 'qzimuostzz.mp4'}, 'acifjvzvpm.mp4': {'label': 'FAKE', 'split': 'train', 'original': 'kbvibjhfzo.mp4'}, 'acqfdwsrhi.mp4': {'label': 'FAKE', 'split': 'train', 'original': 'ccfoszqabv.mp4'}, 'acxnxvbsxk.mp4': {'label': 'FAKE', 'split': 'train', 'original': 'fjlyaizcwc.mp4'}, 'acxwigylke.mp4': {'label': 'FAKE', 'split': 'train', 'original': 'ffcwhpnpuw.mp4'}, 'aczrgyricp.mp4': {'label': 'FAKE', 'split': 'train', 'original': 'slwkmefgde.mp4'}, 'adhsbajydo.mp4': {'label': 'FAKE', 'split': 'train', 'original': 'fysyrqfguw.mp4'}, 'adohikbdaz.mp4': {'label': 'FAKE', 'split': 'train', 'original': 'qjlhemtkx

In [3]:
import os
import numpy as np
import cv2
from tqdm import tqdm  # Import tqdm for the progress bar

def load_video(video_path, max_frames=100, target_size=(224, 224)):
    cap = cv2.VideoCapture(video_path)
    frames = []
    
    while cap.isOpened() and len(frames) < max_frames:
        ret, frame = cap.read()
        if not ret:
            break
        
        frame = cv2.resize(frame, target_size)
        frame = frame / 255.0  # Normalize frame values to [0, 1]
        frames.append(frame)
    
    cap.release()
    
    # If the video has fewer frames than max_frames, pad it with black frames
    while len(frames) < max_frames:
        frames.append(np.zeros((target_size[0], target_size[1], 3)))  # Padding with black frames (RGB)
    
    return np.array(frames)

def pad_sequences(sequences, maxlen=None, padding_value=0):
    if maxlen is None:
        maxlen = max(len(seq) for seq in sequences)
    
    padded_sequences = np.full((len(sequences), maxlen, *sequences[0].shape[1:]), padding_value)
    
    for i, seq in enumerate(sequences):
        padded_sequences[i, :len(seq)] = seq
    
    return padded_sequences

def process_videos_with_metadata(video_dir, metadata, max_frames=100, target_size=(224, 224)):
    videos = []
    labels = []
    additional_features = []

    video_files = os.listdir(video_dir)

    # Use tqdm to add a progress bar to the loop
    for video_file in tqdm(video_files, desc="Processing videos", unit="video"):
        video_path = os.path.join(video_dir, video_file)
        
        # Extract metadata for the video
        video_id = os.path.splitext(video_file)[0]
        video_metadata = metadata.get(video_id, {})
        
        frames = load_video(video_path, max_frames, target_size)
        videos.append(frames)
        labels.append(video_metadata.get('label', 0))  # Default to 0 if no label
        additional_features.append(video_metadata.get('features', {}))  # Adjust based on your metadata structure

    # Pad all sequences to ensure they have the same shape
    videos_padded = np.array(videos)  # Since padding was handled during loading, this is directly converted
    
    return videos_padded, np.array(labels), additional_features

# Function to load metadata (assuming JSON-like structure)
def load_metadata(metadata_path):
    # Load metadata from a file (replace with actual loading logic)
    import json
    with open(metadata_path, 'r') as f:
        metadata = json.load(f)
    return metadata

# Example usage
video_dir = '/Users/lakshya/Desktop/Projects/VeriFace/Dataset/Train Videos'
metadata = load_metadata(metadata_path)

videos, labels, additional_features = process_videos_with_metadata(video_dir, metadata)


Processing videos:   3%|▎         | 13/401 [00:08<04:27,  1.45video/s]


KeyboardInterrupt: 

*Data Extraction*
1. Video Extraction

In [ ]:
import cv2
import os
from tqdm import tqdm  # Import tqdm for progress bar

def extract_frames(video_path, output_dir, frame_rate=1):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    cap = cv2.VideoCapture(video_path)
    count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        if count % frame_rate == 0:
            cv2.imwrite(os.path.join(output_dir, f"frame_{count}.jpg"), frame)
        count += 1
    cap.release()

def process_videos(video_dir, output_base_dir, frame_rate=1):
    if not os.path.exists(output_base_dir):
        os.makedirs(output_base_dir)
    
    # Get all video files in the specified directory
    video_files = [f for f in os.listdir(video_dir) if f.endswith(('.mp4', '.avi', '.mov'))]
    
    # Limit to 400 videos
    video_files = video_files[:400]
    
    # Use tqdm to show progress bar for the videos being processed
    for video_file in tqdm(video_files, desc="Processing videos", unit="video"):
        video_path = os.path.join(video_dir, video_file)
        output_dir = os.path.join(output_base_dir, os.path.splitext(video_file)[0])
        extract_frames(video_path, output_dir, frame_rate)

# Example usage
process_videos(videos, 'Extracted Video')

2. Audio Extraction

In [ ]:
from pydub import AudioSegment
import os
from tqdm import tqdm  # Import tqdm for the progress bar

def extract_audio(video_path, output_audio_path):
    audio = AudioSegment.from_file(video_path, format="mp4")
    audio.export(output_audio_path, format="wav")

def process_audios(video_dir, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Get all video files in the specified directory
    video_files = [f for f in os.listdir(video_dir) if f.endswith(('.mp4', '.avi', '.mov'))]
    
    # Limit to 400 videos
    video_files = video_files[:400]
    
    # Initialize the progress bar
    with tqdm(total=len(video_files), desc="Processing Videos", unit="file") as pbar:
        for video_file in video_files:
            video_path = os.path.join(video_dir, video_file)
            output_audio_path = os.path.join(output_dir, os.path.splitext(video_file)[0] + '.wav')
            extract_audio(video_path, output_audio_path)
            
            # Update the progress bar after processing each file
            pbar.update(1)

# Example usage
process_audios(videos, 'Extracted Audio')

*Preprocessing*
1. Video Preprocessing: Resizing, Normalizing, Augmentation

In [20]:
import tensorflow as tf

def preprocess_video_frame(frame, target_size=(224, 224)):
    frame = tf.image.resize(frame, target_size)
    frame = tf.cast(frame, tf.float32) / 255.0  # Normalize
    return frame


2. Audio Preprocessing: Resampling, Noise reduction

In [21]:
import librosa

def preprocess_audio(audio_path, target_sr=16000):
    y, sr = librosa.load(audio_path, sr=target_sr)
    y = librosa.effects.trim(y)[0]  # Trim silence
    return y

*Model Creation*
1. Video Models: ResNet, VGG-16, C3D, TCN

In [22]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50

def create_resnet_model(input_shape=(224, 224, 3)):
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
    base_model.trainable = False
    model = tf.keras.Sequential([
        base_model,
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax') 
    ])
    return model


resnet_model = create_resnet_model()
resnet_model.save('saved_model.keras')
resnet_model = tf.keras.models.load_model('saved_model.keras')

In [23]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers, models

def create_vgg16_model(input_shape=(224, 224, 3), num_classes=10):
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
    base_model.trainable = False  

    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(256, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax')  
    ])

    return model


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

def create_c3d_model(input_shape=(16, 112, 112, 3), num_classes=10):
    model = models.Sequential()

    # Block 1
    model.add(layers.Conv3D(64, kernel_size=(3, 3, 3), activation='relu', padding='same', input_shape=input_shape))
    model.add(layers.MaxPooling3D(pool_size=(1, 2, 2), padding='same'))

    # Block 2
    model.add(layers.Conv3D(128, kernel_size=(3, 3, 3), activation='relu', padding='same'))
    model.add(layers.MaxPooling3D(pool_size=(2, 2, 2), padding='same'))

    # Block 3
    model.add(layers.Conv3D(256, kernel_size=(3, 3, 3), activation='relu', padding='same'))
    model.add(layers.Conv3D(256, kernel_size=(3, 3, 3), activation='relu', padding='same'))
    model.add(layers.MaxPooling3D(pool_size=(2, 2, 2), padding='same'))

    # Block 4
    model.add(layers.Conv3D(512, kernel_size=(3, 3, 3), activation='relu', padding='same'))
    model.add(layers.Conv3D(512, kernel_size=(3, 3, 3), activation='relu', padding='same'))
    model.add(layers.MaxPooling3D(pool_size=(2, 2, 2), padding='same'))

    # Block 5
    model.add(layers.Conv3D(512, kernel_size=(3, 3, 3), activation='relu', padding='same'))
    model.add(layers.Conv3D(512, kernel_size=(3, 3, 3), activation='relu', padding='same'))
    model.add(layers.MaxPooling3D(pool_size=(2, 2, 2), padding='same'))

    # Fully connected layers
    model.add(layers.Flatten())
    model.add(layers.Dense(4096, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(4096, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(num_classes, activation='softmax'))

    return model


In [25]:
# Video Feature Extraction - TCN using Conv2D for image input
def create_tcn_model():
    model = Sequential()
    model.add(layers.Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same', input_shape=(224, 224, 3)))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(layers.Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(layers.Flatten())
    return model

2. Audio Models: Wav2Vec, CRNN, VGGish, WaveNet

In [ ]:
from transformers import Wav2Vec2Processor, Wav2Vec2Model
import torch
import librosa

# Load pre-trained Wav2Vec2.0 model and processor
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base-960h")

# Load and preprocess audio
def extract_wav2vec2_features(audio_path):
    # Load audio file with librosa
    y, sr = librosa.load(audio_path, sr=16000)  # Wav2Vec2 requires 16kHz sample rate
    
    # Preprocess the audio to match the input format for Wav2Vec2.0
    input_values = processor(y, return_tensors="pt", sampling_rate=sr).input_values
    
    # Extract features (output from the last hidden layer)
    with torch.no_grad():
        features = model(input_values).last_hidden_state
    
    return features

wav2vec_features = extract_wav2vec2_features('Extracted Audio')

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
import librosa
import numpy as np


def extract_mel_spectrogram(audio_path, n_mels=128):
    y, sr = librosa.load(audio_path, sr=None)
    mel_spec = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=n_mels)
    mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)
    return mel_spec_db

# CRNN model
def create_crnn_model(input_shape=(128, 128, 1), num_classes=10):
    model = models.Sequential()
    
    # CNN layers
    model.add(layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    
    model.add(layers.Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    
    model.add(layers.Conv2D(128, kernel_size=(3, 3), activation='relu'))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))

    # RNN layers (GRU or LSTM)
    model.add(layers.Reshape(target_shape=(-1, 128)))
    model.add(layers.GRU(128, return_sequences=False))
    
    # Dense layers
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(num_classes, activation='softmax'))
    
    return model



In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import soundfile as sf
import librosa

# Load the VGGish model from TensorFlow Hub
def load_vggish_model():
    vggish_model = hub.load("https://tfhub.dev/google/vggish/1")
    return vggish_model

# Preprocess the audio file to match VGGish input requirements (16kHz, mono)
def preprocess_audio(audio_path):
    # Load audio using soundfile and librosa
    audio, sr = librosa.load(audio_path, sr=16000)  # Resample to 16kHz if not already
    # VGGish expects mono audio (single channel), ensure it's mono
    if len(audio.shape) > 1:
        audio = np.mean(audio, axis=1)
    return audio

# Extract features using VGGish model
def extract_vggish_features(audio_path, vggish_model):
    audio = preprocess_audio(audio_path)
    # Convert the audio to a tensor with the correct shape for VGGish
    audio_input = tf.convert_to_tensor(audio, dtype=tf.float32)
    audio_input = tf.expand_dims(audio_input, 0)  # Add batch dimension
    # Pass through VGGish model
    features = vggish_model(audio_input)
    return features


vggish_model = load_vggish_model()
audio_path = 'Extracted Audio' 
vggish_features = extract_vggish_features(audio_path, vggish_model)


In [ ]:
from tensorflow.keras import layers, models

def residual_block(x, dilation_rate):
    conv = layers.Conv1D(filters=64, kernel_size=2, padding='causal', dilation_rate=dilation_rate)(x)
    conv = layers.Activation('relu')(conv)
    conv = layers.Conv1D(filters=64, kernel_size=2, padding='causal')(conv)
    
    # Residual connection
    x = layers.add([x, conv])
    return x

def create_wavenet_model(input_shape=(16000, 1), num_classes=10):
    inputs = layers.Input(shape=input_shape)
    
    x = layers.Conv1D(filters=64, kernel_size=2, padding='causal')(inputs)
    
    # Stack of residual blocks with increasing dilation rates
    for dilation_rate in [1, 2, 4, 8, 16]:
        x = residual_block(x, dilation_rate)
    
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    
    model = models.Model(inputs, outputs)
    return model



*Feature Extraction*
1. Video 

In [ ]:
def extract_vgg16_features(video_frames):
    vgg16_model = create_vgg16_model()
    vgg16_features = vgg16_model.predict(video_frames)
    return vgg16_features

def extract_resnet50_features(video_frames):
    resnet_model = create_resnet_model()
    resnet_features = resnet_model.predict(video_frames)
    return resnet_features

def extract_c3d_features(video_frames):
    c3d_model = create_c3d_model()
    c3d_features = c3d_model.predict(video_frames)
    return c3d_features

def extract_tcn_features(mel_spectrogram):
    tcn_model = create_tcn_model()
    tcn_features = tcn_model.predict(mel_spectrogram)
    return tcn_features

2. Audio

In [ ]:
def extract_crnn_features(mel_spectrogram):
    crnn_model = create_crnn_model()
    crnn_features = crnn_model.predict(mel_spectrogram)
    return crnn_features

def extract_wavenet_features(audio_data):
    wavenet_model = create_wavenet_model()
    wavenet_features = wavenet_model.predict(audio_data)
    return wavenet_features

*Feature Fusion*
1. Video 


In [ ]:
from tensorflow.keras.layers import concatenate

# Fuse video features
def fuse_video_features(vgg16_features, resnet50_features, tcn_features, c3d_features):
    return layers.Concatenate()([vgg16_features, resnet50_features, tcn_features, c3d_features])

2. Audio

In [ ]:
from tensorflow.keras.layers import concatenate

def fuse_audio_features(crnn_features, wav2vec_features, vggish_features, wavenet_features):
    return layers.Concatenate()([crnn_features, wav2vec_features, vggish_features, wavenet_features])

*Final Model*

In [ ]:
# Final Model combining both video and audio
def create_final_model(video_shape, audio_shape, num_classes=10):
    tcn_model = create_tcn_model()
    c3d_model = create_c3d_model()
    vgg16_model = create_vgg16_model()
    
    wavenet_model = create_wavenet_model()
    crnn_model = create_crnn_model()  
    
    
    video_input = layers.Input(shape=video_shape)
    audio_input = layers.Input(shape=audio_shape)

    mel_spectrogram = extract_mel_spectrogram(audio_input, n_mels=128)

    # Video Features
    vgg16_features = extract_vgg16_features('Extracted Video')
    resnet50_features = extract_resnet50_features('Extracted Video')
    tcn_features = extract_tcn_features(mel_spectrogram)
    c3d_features = extract_c3d_features('Extracted Video')
    
    # Audio Features
    crnn_features =  extract_crnn_features(mel_spectrogram)
    wavnet_features = extract_wavenet_features('Extracted Audio')

    fused_video_features = fuse_video_features(vgg16_features, resnet50_features, tcn_features, c3d_features)
    fused_audio_features = fuse_audio_features(crnn_features, wav2vec_features, vggish_features, wavnet_features)

    combined_features = layers.Concatenate()([fused_video_features, fused_audio_features])

    x = layers.Dense(256, activation='relu')(combined_features)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    model = models.Model(inputs=[video_input, audio_input], outputs=outputs)
    return model

# Compile and train the model (assuming data is preprocessed and ready)
video_shape = (224, 224, 3)
audio_shape = (16000, 1)
num_classes = 10

model = create_final_model(video_shape, audio_shape, num_classes)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Saving Model

In [ ]:
# To train the model:
history = model.fit([X_train_video, X_train_audio], y_train, batch_size=32, epochs=10, validation_split=0.2)

In [ ]:
#Save the trained model
model.save('final_model.h5')